In [1]:
import os
import modin.pandas as pd; pd.set_option(
    'display.float_format', '{:.5f}'.format)
import numpy as np
import matplotlib.pyplot as plt
import time; start_time = time.time()
import warnings; warnings.filterwarnings(
    "ignore", message=".*defaulting to pandas implementation.*")

# Import Model

In [2]:
models_path = os.path.join(os.getcwd(),'trained_models')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

0   2024_10_07 17-55-55 Deep Neural Network 3oosRMSE
1   2024_10_08 00-00-41 Deep Neural Network 3oosRMSE
2   2024_10_08 00-24-59 Deep Neural Network 3oosRMSE
3   2024_10_08 00-42-22 Deep Neural Network 3oosRMSE
4   2024_10_09 09-01-43 Deep Neural Network 6oosRMSE
5   2024_10_09 09-08-27 Deep Neural Network 9oosRMSE
6   2024_10_09 09-11-43 Deep Neural Network 11oosRMSE
7   2024_10_09 10-09-20 Deep Neural Network 15oosRMSE
8   2024_10_09 11-11-13 Deep Neural Network 16oosRMSE


In [ ]:
model = models[0]
model

In [ ]:
model_path = os.path.join(models_path,model)

In [ ]:
model_files = os.listdir(model_path)
model_files

In [ ]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [ ]:
with open(spec_path, "r") as specs:
    for line in specs:
        print(line.strip())

In [ ]:
model_files

In [ ]:
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outofsample_results') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]
outsample

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
plt.figure(figsize=(4, 3),dpi=80)
for b in barrier_type_names:
    plt.plot(MAE.loc[:, b], label=b)
plt.xlabel('maturity in days')
plt.ylabel('RMSE')
plt.legend()
plt.show()
plt.close()

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    errors['RMSE'] = RMSE

    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    errors['MAE'] = MAE

    errors_bypayoff[w] = errors

In [ ]:
for w, errors in errors_bypayoff.items():
    print("RMSE:\n")
    print(errors['RMSE'])
    print()

In [ ]:
plt.figure(figsize=(4, 3),dpi=80)
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['RMSE'][b]
        plt.plot(
            MAE, label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.ylabel('RMSE')
plt.xlabel('maturity in days')
plt.show()
plt.close()

In [ ]:
for w, errors in errors_bypayoff.items():
    print("MAE:\n")
    print(errors['MAE'])
    print()

In [ ]:
plt.figure(figsize=(4, 3),dpi=80)
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['MAE'][b]
        plt.plot(
            MAE, label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.ylabel('MAE')
plt.xlabel('maturity in days')
plt.show()
plt.close()

In [ ]:
end_time = time.time()
runtime = end_time - start_time
print(f"cpu: {runtime}")
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")